In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

apt_tx_indi_26_weeks = pd.read_csv('recent26_tx_indi.csv')
apt_res_indi_26_weeks = pd.read_csv('recent26_res_indi.csv')

# 데이터 전처리
goyang_monthly_tx = apt_tx_indi_26_weeks.copy()
goyang_monthly_res = apt_res_indi_26_weeks.copy()

# 'CLS_NM' 열에서 특정 값이 있는 행들만 필터링
''' region의 '서울' 대신에 비교할 지역을 넣어주시면 됩니다.
지역 이름은 엑셀파일 recent36_res_indi(전세가격지수), recent26_tx_indi(매매가격지수)의 CLS_NM 열에서 확인 가능합니다.'''
region = '서울'
'''매매가격지수 관련 데이터 파싱'''
filtered_data1 = goyang_monthly_tx[goyang_monthly_tx['CLS_NM'].isin(['전국', region])]
filtered_data2 = goyang_monthly_tx[goyang_monthly_tx['CLS_NM'].isin(['6대광역시', region])]
'''전세가격지수 관련 데이터 파싱'''
filtered_data3 = goyang_monthly_res[goyang_monthly_res['CLS_NM'].isin(['전국', region])]
filtered_data4 = goyang_monthly_res[goyang_monthly_res['CLS_NM'].isin(['6대광역시', region])]

def convert_to_year_week(week_code):
    year = int(week_code[:4])
    week = int(week_code[4:])
    return f'{year}년 {week}주차'

filtered_data1['WRTTIME_IDTFR_ID'] = filtered_data1['WRTTIME_IDTFR_ID'].astype(str)
filtered_data1['formatted_week'] = filtered_data1['WRTTIME_IDTFR_ID'].apply(convert_to_year_week)
filtered_data2['WRTTIME_IDTFR_ID'] = filtered_data2['WRTTIME_IDTFR_ID'].astype(str)
filtered_data2['formatted_week'] = filtered_data2['WRTTIME_IDTFR_ID'].apply(convert_to_year_week)
filtered_data3['WRTTIME_IDTFR_ID'] = filtered_data3['WRTTIME_IDTFR_ID'].astype(str)
filtered_data3['formatted_week'] = filtered_data3['WRTTIME_IDTFR_ID'].apply(convert_to_year_week)
filtered_data4['WRTTIME_IDTFR_ID'] = filtered_data4['WRTTIME_IDTFR_ID'].astype(str)
filtered_data4['formatted_week'] = filtered_data4['WRTTIME_IDTFR_ID'].apply(convert_to_year_week)

'''1번그래프 : 전국과 매매가격지수 비교'''

# y축 최솟값과 최댓값 계산
y_min1 = filtered_data1['DTA_VAL'].min()
y_min_with_margin1 = y_min1 - (y_min1 * 0.01)
y_max1 = filtered_data1['DTA_VAL'].max()
y_max_with_margin1 = y_max1 + (y_max1 * 0.01)

# Plotly 그래프 생성
fig1 = px.line(filtered_data1,
               x='formatted_week',
               y='DTA_VAL',
               color='CLS_NM',
               color_discrete_map={
                    region: '#0047AB',
                   '6대광역시': '#D50032',
                   '전국': '#D50032',
               },
               markers=False,
               line_shape='spline',
               )

# 선 그래프 색상 및 굵기 설정
fig1.update_traces(line=dict(width=6))

# y축 라벨 형식 수정
tickvals1 = list(range(int(y_min_with_margin1), int(y_max_with_margin1) + 1))

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig1.update_layout(
    title={
        'text': '매매가격지수 전국과 비교',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 25, 'family': 'Montserrat', 'color': '#2B3E50'}
    },
    legend=dict(
        font=dict(size=30),  # 범례 텍스트 크기 조정
        title=None,  # 범례 제목 숨기기
    ),
    xaxis=dict(
        showline=True,
        showticklabels=True,
        tickfont=dict(size=15, family='Verdana', color='gray'),
        title=None,
        showgrid=True,  # 세로선 표시
        gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색
    ),
    yaxis=dict(
        showline=True,
        showticklabels=True,
        title=None,
        tickfont=dict(size=25, family='Verdana', color='gray'),
        tickmode='array',  # 수동으로 tick 설정
        tickvals=tickvals1,  # y축 눈금 위치
        showgrid=True,  # 가로선 표시
        gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색
        gridwidth=1,
        range=[y_min_with_margin1, y_max_with_margin1]  # y축의 최솟값과 최댓값에 마진 추가
    ),
    width=1120,  # 그래프 너비 (16:9 비율로 설정)
    height=630,  # 그래프 높이 (16:9 비율로 설정)
    plot_bgcolor='white',  # 그래프 배경색을 깔끔한 흰색으로
    paper_bgcolor='white',  # 전체 배경색을 가벼운 회색
    xaxis_tickformat='%Y년%주차',
    xaxis_tickangle=45,  # x축 눈금 기울기
    margin=dict(l=40, r=40, t=80, b=60),  # 여백 조정
    annotations=[
        dict(
            text="출처:한국부동산원 부동산통계정보",
            x=1,  # 오른쪽 끝으로 배치
            y=0,  # 아래쪽 끝으로 배치
            xref="paper",  # x축 기준으로 paper 사용
            yref="paper",  # y축 기준으로 paper 사용
            xanchor='right',  # 오른쪽 끝에 맞춤
            yanchor='bottom',  # 아래쪽 끝에 맞춤
            showarrow=False,  # 화살표 없이 텍스트만 표시
            font=dict(size=12, color="gray")  # 작은 회색 글씨로 표시
        )
    ]
)

'''2번그래프 : 6대광역시와 매매가격지수 비교'''

# y축 최솟값과 최댓값 계산
y_min2 = filtered_data2['DTA_VAL'].min()
y_min_with_margin2 = y_min2 - (y_min2 * 0.008)
y_max2 = filtered_data2['DTA_VAL'].max()
y_max_with_margin2 = y_max2 + (y_max2 * 0.008)

# Plotly 그래프 생성
fig2 = px.line(filtered_data2,
               x='formatted_week',
               y='DTA_VAL',
               color='CLS_NM',
               color_discrete_map={
                   region: '#0047AB',
                   '6대광역시': '#D50032',
                   '전국': '#D50032',
               },
               markers=False,
               line_shape='spline',
               )

# 선 그래프 색상 및 굵기 설정
fig2.update_traces(line=dict(width=6))

# y축 라벨 형식 수정
tickvals2 = list(range(int(y_min_with_margin2), int(y_max_with_margin2) + 1))

# 그래프 레이아웃 업데이트 (크기, 제목, 축 스타일, 배경색 등)
fig2.update_layout(
    title={
        'text': '매매가격지수 최근 26주 고양시 6대광역시 비교',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 25, 'family': 'Montserrat', 'color': '#2B3E50'}
    },
    legend=dict(
        font=dict(size=30),  # 범례 텍스트 크기 조정
        title=None,  # 범례 제목 숨기기
    ),
    xaxis=dict(
        showline=True,
        showticklabels=True,
        tickfont=dict(size=15, family='Verdana', color='gray'),
        title=None,
        showgrid=True,  # 세로선 표시
        gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색
    ),
    yaxis=dict(
        showline=True,
        showticklabels=True,
        title=None,
        tickfont=dict(size=25, family='Verdana', color='gray'),
        tickmode='array',  # 수동으로 tick 설정
        tickvals=tickvals2,  # y축 눈금 위치
        showgrid=True,  # 가로선 표시
        gridcolor='rgba(211, 211, 211, 0.5)',  # 연한 회색 점선
        gridwidth=1,
        range=[y_min_with_margin2, y_max_with_margin2]  # y축의 최솟값과 최댓값에 마진 추가
    ),
    width=1120,  # 그래프 너비 (16:9 비율로 설정)
    height=630,  # 그래프 높이 (16:9 비율로 설정)
    plot_bgcolor='white',  # 그래프 배경색을 깔끔한 흰색으로
    paper_bgcolor='white',  # 전체 배경색을 가벼운 회색
    xaxis_tickformat='%Y년%주차',
    xaxis_tickangle=45,  # x축 눈금 기울기
    margin=dict(l=40, r=40, t=80, b=60),  # 여백 조정
    annotations=[
        dict(
            text="출처:한국부동산원 부동산통계정보",
            x=1,  # 오른쪽 끝으로 배치
            y=0,  # 아래쪽 끝으로 배치
            xref="paper",  # x축 기준으로 paper 사용
            yref="paper",  # y축 기준으로 paper 사용
            xanchor='right',  # 오른쪽 끝에 맞춤
            yanchor='bottom',  # 아래쪽 끝에 맞춤
            showarrow=False,  # 화살표 없이 텍스트만 표시
            font=dict(size=12, color="gray")  # 작은 회색 글씨로 표시
        )
    ]
)